In [1]:
import pandas as pd
import sqlite3

In [2]:
def get_age_at_first_race(conn: str)-> pd.DataFrame:
    """
    Calculate how old were the drivers when they first compeated.

    :param conn: The database connection.
    :return: A DataFrame containing driverId, and age_at_first_race columns.
    """
    sql_query = '''
    WITH DriverFirstRace AS (
        SELECT
            r.driverId,
            MIN(ra.year) AS first_race_year
        FROM
            results r
        JOIN
            races ra ON r.raceId = ra.raceId
        WHERE
            ra.year <> 2023
        GROUP BY
            r.driverId
    )
    SELECT
        d.driverId,
        ra.year - CAST(strftime('%Y', d.dob) AS INTEGER) AS age_at_first_race
    FROM
        drivers d
    JOIN
        DriverFirstRace df ON d.driverId = df.driverId
    JOIN
        races ra ON df.first_race_year = ra.year
    GROUP BY
        d.driverId;
    '''
    df = pd.read_sql_query(sql_query, conn)

    return df

In [ ]:
def get_age_at_each_race(conn: str) -> pd.DataFrame:
    """
    Calculate the age of drivers for each race year.

    :param conn: The database connection.
    :return: A DataFrame containing driverId, race_year, and age_at_race columns.
    """
    sql_query = '''
    SELECT
        r.driverId,
        ra.year AS year,
        ra.year - CAST(strftime('%Y', d.dob) AS INTEGER) AS age_at_race
    FROM
        results r
    JOIN
        races ra ON r.raceId = ra.raceId
    JOIN
        drivers d ON r.driverId = d.driverId
    WHERE
        ra.year <> 2023;
    '''
    df = pd.read_sql_query(sql_query, conn)

    return df